# How to build Angular components?

How to create a mock filesystem?


In [1]:
var execSync = require('child_process').execSync;
// first install angular cli
var path = require('path');
var data = {
    'angular-cli.json': 
`
{
    "project": {
        "name": "act-ecommerce"
    },
    "apps": [
        {
            "root": "src",
            "main": "main.ts",
            "tsconfig": "../tsconfig.json",
            "prefix": "bc",
            "styles": [],
            "scripts": []
        }
    ]
}
`
};




Run angular-cli commands from nodejs?

Add a component to the specified project using angular-cli?



In [2]:
var execSync = require('child_process').execSync;
try {
    require.resolve('@angular/cli');
} catch (e) {
    execSync('npm install @angular/cli');
}
var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE;

// use utility/filesystem to mock all fs and typescript commands
var mockTypescriptFs, mock, cli;

var importer = require('../Core');
// call the CLI just like from command line
var ng = (project, args = ['generate', 'component', 'test']) => {
    // set up project path
    var previous = process.cwd();
    
    var conf = {
        cliArgs: args,
        inputStream: null,
        outputStream: process.stdout
    };
    
    // execute
    return importer.interpretAll('memory-fs rewire')
        .then(r => {
            //if(typeof cli == 'undefined') {
                mockTypescriptFs = eval("'use strict';" + r[0].code);
                // overlay out temp filesystem on top of current filesystem
                mock = mockTypescriptFs(data);
                cli = require('@angular/cli');
            //}
            process.chdir(project);
            return cli(conf);
        })
        // ng is mocked at this point
        .then(() => {
            process.chdir(previous);
            //mock.stopAll();
            data = mock.data();
            return mock.data();
        });
}
(ng);



[Function: ng]

List projects by name?



In [3]:
var execSync = require('child_process').execSync();
try {
    require.resolve('glob');
} catch (e) {
    execSync('glob');
}
var glob = require('glob');

// But we also want to automatically load projects?
var listProjects = (root = PROFILE_PATH) => {
    return new Promise((resolve, reject) => {
        glob('**/package.json', {
            ignore: ['**/node_modules/**',
                    '**/packages/**',
                    '**/vendor/**',
                    '/Users/**/Downloads/**',
                    '/Users/**/Library/**',
                    '/Users/**/Applications/**',
                    '/Users/**/Music/**',
                    '**/wp-content/**',
                    '**/wp-includes/**',
                    '**/Pods/**',
                    '**/svn/**',
                    '**/.git/**',
                    '**/.vscode/**',
                    '**/.npm/**',
                    '**/Cache/**',
                    '**/Creative Cloud Files/**'],
            cwd: path.resolve(root),
            silent: true,
            nodir: true,
            strict: false
        }, (err, matches) => {
            var result = {};
            matches.forEach(m => {
                var projectPath = path.dirname(path.join(root, m));
                result[path.basename(projectPath)] = projectPath;
            })
            return resolve(result);
        });
    });
};
(listProjects);



[Function: listProjects]

Build angular modules from nodejs?



In [4]:
// Wouldn't it be nice if we could list these projects as interactive icons?
var fs = require('fs');
var convertNgUniversal = (project) => {
    return ng(project, ['generate', 'module', '--spec', '--flat', 'AppServer'])
    .then(() => {
        var serverModulePath = project + '/src/app/app-server.module.ts';
        var app_server = memfs.readFileSync(serverModulePath).toString();
        app_server = app_server.replace('CommonModule', `ServerModule,
AppModule
],
bootstrap: [
	  AppComponent
`);
        memfs.writeFileSync(serverModulePath, app_server);


        var appModulePath = project + '/src/app/app.module.ts';
        var app_module = fs.readFileSync(appModulePath).toString();
        app_module = app_module.replace('BrowserModule,',` BrowserModule.withServerTransition({
            appId: 'universal-demo-app'
        }),
`);
        return importer.getCells('./angular universal.ipynb')
    })
    .then(cells => {
        var appServerPath = project + '/src/main.server.ts';
        memfs.writeFileSync(appServerPath, cells.join('\n'));
    })
};
(convertNgUniversal);



[Function: convertNgUniversal]

Compile the project if needed?

In [5]:
var execSync = require('child_process').execSync;
try {
    require.resolve('webpack');
    require.resolve('@ngtools/webpack');
} catch (e) {
    execSync('npm install webpack @ngtools/webpack');
}

// source https://github.com/christianalfoni/webpack-bin/issues/106
var webpackAngularProject = (project, memfs) => {
    // set up project path
    var previous = process.cwd();
    process.chdir(project);
    module.paths.unshift(project + '/node_modules');
    
    var webpack = require('webpack');
    var ngtools = require('@ngtools/webpack');

    var conf = {
        entry: {
            main: './src/main.server.ts'
        },
        resolve: {
          extensions: ['.ts', '.js']
        },
        target: 'node',
        output: {
            path: process.cwd() + '/dist',
            filename: '[name].js'
        },
        plugins: [
            new ngtools.AotPlugin({
                tsConfigPath: process.cwd() + '/tsconfig.json',
            })
        ],
        module: {
            rules: [
                {
                  test: /\.ts$/,
                  loader: '@ngtools/webpack',
                }
            ]
        }
    }

    var compiler = webpack(conf);

    compiler.inputFileSystem = memfs;
    compiler.outputFileSystem = memfs;
    compiler.resolvers.normal.fileSystem = memfs;
    compiler.resolvers.context.fileSystem = memfs;

    return new Promise((resolve, reject) => {
        compiler.run(function(err, stats) {
            process.chdir(previous);
            if (err) reject(err);
            resolve(eval(stats.compilation.assets['main.js'].source()));
        });
    });
}
(webpackAngularProject);


[Function: webpackAngularProject]

Display angular modules in jupyter?



In [6]:
$$.async();
var execSync = require('child_process').execSync;
try {
    require.resolve('npm');
} catch (e) {
    execSync('npm install npm');
}

/*
            return listProjects(project);
        })
        .then(r => {
            project = r[path.basename(project)];
*/
var project = '~/Documents/new-portal';
project = path.resolve(project.replace(/^~\//ig, PROFILE_PATH + '\/'));
ng(path.dirname(project), ['new', '--skip-git', 'new-portal'])
    .then(() => ng(project, [
    'generate', 'module', '--spec', '--flat', 'AppServer']))
    /*

    
            memfs.mkdirpSync(project);
            memfs.writeFileSync(
                path.join(project, 'angular-cli.json'),
                data['angular-cli.json']);

    */

    .then(() => {
        // set up project path
        var previous = process.cwd();
        process.chdir(project);
        return new Promise((resolve, reject) => {
            var npm = require('npm');

            npm.load(function(err) {
                // handle errors
                if(err) {
                    return reject(err);
                }

                console.log(project);
                // install module ffi
                npm.commands.init([], function(er, data) {
                    process.chdir(previous);
                    if(err) {
                        reject(err);
                    }
                    // log errors or data
                    resolve(data);
                });

                npm.on('log', function(message) {
                    // log installation progress
                    console.log(message);
                });
            });
        });
    })
    //convertNgUniversal(project)
    //    .then(() => ng(project, ['build', '--aot', '--prod']))
    //.then(() => webpackAngularProject(project, memfs))
    //.then(() => execCmd())
    .then(r => {
        var folder = data;
        path.resolve(project)
            .split(path.sep)
            .filter((e, i) => i > 0 || e !== '')
            .forEach(p => folder = folder[p]);
        return folder;
    })
    .catch(e => console.log(e))
    .then(r => $$.sendResult(r))


installing ng
  create .editorconfig
  create README.md
  create src\app\app.component.css
  create src\app\app.component.html
  create src\app\app.component.spec.ts
  create src\app\app.component.ts
  create src\app\app.module.ts
  create src\assets\.gitkeep
  create src\environments\environment.prod.ts
  create src\environments\environment.ts
  create src\favicon.ico
  create src\index.html
  create src\main.ts
  create src\polyfills.ts
  create src\styles.css
  create src\test.ts
  create src\tsconfig.app.json
  create src\tsconfig.spec.json
  create src\typings.d.ts
  create .angular-cli.json
  create e2e\app.e2e-spec.ts
  create e2e\app.po.ts
  create e2e\tsconfig.e2e.json
  create karma.conf.js
  create package.json
  create protractor.conf.js
  create tsconfig.json
  create tslint.json
Installing packages for tooling via npm.
Installed packages for tooling via npm.
Project 'new-portal' successfully created.
Already imported
[ 'import notebook.ipynb', 'interpret all notebooks.ipy